In [ ]:
import numpy as np
import pandas as pd
import random as r
%matplotlib inline
import matplotlib.pyplot as plt # side-stepping mpl backend
from scipy.stats import entropy


def render_output():
    df = pd.read_table('perf/dump.tsv')

    O=df.sum(axis=1)

    print(f'entropy: {entropy(O)}')

    fig, axs = plt.subplots(2, figsize = (20,10), sharex=True)
    axs[1].autoscale(enable=True, axis='x', tight=True)
    axs[0].plot(O, lw=0.1)
    axs[1].imshow(df.T, interpolation='none', aspect='auto')
    fig.tight_layout()
    plt.show()

In [ ]:
import subprocess

pipe = subprocess.PIPE
result = subprocess.run(['go', 'run', 'main.go'], cwd='perf', stdout=pipe, stderr=pipe)

if result.returncode==0:
    print(result.stdout.decode('utf-8'))
    render_output()
else:
    print(result.stderr.decode('utf-8'))

In [ ]:
render_output()